# Bronze To Sliver

In [0]:
from pyspark.sql.functions import col,sum

In [0]:
##reading from bronze layer
bronzeloaction="/mnt/bayerhackathon/squad2-srinivasu/bronze/"
customerDf=spark.read.format("delta").load(f"{bronzeloaction}/customer/")
customerDf_col=customerDf.filter("phone is not null").dropDuplicates()



orderDf=spark.read.format("delta").load(f"{bronzeloaction}/order/").dropDuplicates()

oderDf_data=customerDf_col.alias("cust")\
                    .join(orderDf.alias("ord"),col("cust.customer_id")==col("ord.customer_id"),"inner")\
                    .selectExpr("ord.*")



orderlineDf=spark.read.format("delta").load(f"{bronzeloaction}/orderline/").dropDuplicates()

orderlineDf_data=orderlineDf.alias("ord_line")\
                     .join(oderDf_data.alias("ord_dt"),col("ord_line.order_id")==col("ord_dt.order_id"),"inner")\
                     .selectExpr("ord_line.*")   

orderdf_tot=orderlineDf_data.withColumn("total_purchase_value_new",col("quantity")*col("price"))\
                            .withColumn("total_purchase_value_new",col("total_purchase_value_new").cast("double"))


orderdf_agg=orderdf_tot.groupBy("order_id").agg(sum("total_purchase_value_new").alias("tot_purchase_vl"))



order_df_slv=orderdf_agg.alias("ord_agg")\
            .join(oderDf_data.alias("ord_dt"),col("ord_agg.order_id")==col("ord_dt.order_id"),"inner")\
            .selectExpr("ord_dt.*","ord_agg.tot_purchase_vl ").drop("total_purchase_value")\
            
order_df_sl_f=order_df_slv.withColumn("total_purchase_value",col("tot_purchase_vl"))\
                          .drop("tot_purchase_vl")          

custBehavDf=spark.read.format("delta").load(f"{bronzeloaction}/customerbehaviour/").dropDuplicates()
custBehavDf_data=customerDf_col.alias("cust")\
                .join(custBehavDf.alias("cust_beh"),col("cust.customer_id")==col("cust_beh.customer_id"),"inner")\
                .selectExpr("cust_beh.*")





In [0]:
customerDf_col.count()

In [0]:
order_df_sl_f.count()

In [0]:
orderlineDf.count()

In [0]:
custBehavDf_data.count()

In [0]:
##writing to sliver location
sliverlocation="/mnt/bayerhackathon/squad2-srinivasu/sliver/"
customerDf_col.write.format("delta").mode("overwrite").save(f"{sliverlocation}/customer/")
order_df_sl_f.write.format("delta").mode("overwrite").save(f"{sliverlocation}/order/")
orderlineDf_data.write.format("delta").mode("overwrite").save(f"{sliverlocation}/orderline/")
custBehavDf_data.write.format("delta").mode("overwrite").save(f"{sliverlocation}/customerbehaviour/")
